In [ ]:
def count_eye_blinks_v2(eeg_epochs, sfreq, channel_names, frontal_channels=['Fz', 'F4']):
    """
    Returns a list of ratio (peak_amp/variance) for each subepoch in all epochs,
    using time-domain peak amplitude.
    """
    frontal_idxs = [i for i, ch in enumerate(channel_names) if ch in frontal_channels]
    if not frontal_idxs:
        raise ValueError("No frontal channels found in data.")

    ratios = []
    peaks = []
    variances = []

    for epoch in eeg_epochs:
        # Each epoch is length 1279 instead of 1280, so zero-pad
        if epoch.shape[1] == 1279:
            epoch = np.pad(epoch, ((0, 0), (0, 1)), mode='constant')
            
        n_samples = epoch.shape[1]
        n_subepochs = 10
        subepoch_len = 128

        # Only process if epoch is long enough
        if n_samples < n_subepochs * subepoch_len:
            continue

        for i in range(n_subepochs):
            start = i * subepoch_len
            end = start + subepoch_len
            sub_epoch = epoch[:, start:end]

            # Data from 2 frontal electrodes
            frontal_avg = np.mean(sub_epoch[frontal_idxs, :], axis=0)

            # Get the time-domain peak amplitude per subepoch
            peak_amp = np.max((frontal_avg))
            peaks.append(peak_amp)

            # Get the variance for the subepoch
            variance = np.std(frontal_avg)
            variances.append(variance)

            # Calculate the ratio between the peak amplitude and variance of the subepoch
            ratio = peak_amp / variance if variance != 0 else np.nan
            ratios.append(ratio)

    return ratios, peaks, variances



In [ ]:
# Get one epoch (e.g., first participant, first stimulus, first epoch)
single_epoch = epochs_organized[0][0][0]  # shape: (n_channels, n_samples)

# Call the function for just this epoch
ratios, peaks, variances = count_eye_blinks_v2(
    eeg_epochs=[single_epoch],  # Pass as a list
    sfreq=eeg_fs,
    channel_names=ch_names
)

print(ratios)  # This will be a list of ratios for each subepoch in that epoch
print(peaks)
print(variances)

In [ ]:
def count_eye_blinks_per_channel(
    eeg_epochs, sfreq, channel_names, frontal_channels,
    z_thresh=2, min_dist_ms=100
):
    """
    Count eye blinks per frontal channel and return z-scored signals for each epoch.

    Returns:
        blink_counts: list of lists of int, blinks per channel per epoch
                      (shape: [n_epochs][n_channels])
        z_scored_epochs: list of np.ndarray, z-scored frontal signals per epoch
                         (shape of each: [n_channels, n_times])
    """

    frontal_idxs = [i for i, ch in enumerate(channel_names) if ch in frontal_channels]
    if not frontal_idxs:
        raise ValueError("No frontal channels found in data.")

    blink_counts = []
    z_scored_epochs = []
    min_dist_samples = sfreq * (min_dist_ms/1000)

    for epoch_idx, epoch in enumerate(eeg_epochs):
        epoch_counts = []
        z_epoch = []

        for ch_local_idx, ch_idx in enumerate(frontal_idxs):
            signal_ch = epoch[ch_idx, :]
            z_data = zscore(signal_ch)
            z_epoch.append(z_data)

            # Detect candidate peaks
            pos_peaks, _ = signal.find_peaks(z_data, height=z_thresh)
            neg_peaks, _ = signal.find_peaks(-z_data, height=z_thresh)

            all_peaks = np.sort(np.concatenate((pos_peaks, neg_peaks)))

            if len(all_peaks) == 1:
                epoch_counts.append(1)

            elif len(pos_peaks) == 1 and len(neg_peaks == 1) and (abs(neg_peaks[0]-pos_peaks[0]) <= min_dist_samples):
                epoch_counts.append(1)

            else:# Apply global min_distance across all peak types
                filtered_peaks = []

                last_peak = -100000
                for idx, peak in enumerate(all_peaks):
                    if abs(last_peak-peak) >= min_dist_samples :
                        filtered_peaks.append(peak)
                        last_peak = peak
                    else:
                        last_peak = peak

                epoch_counts.append(len(filtered_peaks))

        blink_counts.append(epoch_counts)
        z_scored_epochs.append(np.array(z_epoch))

    return blink_counts, z_scored_epochs


In [ ]:
frontal_ch = ['Fz', 'F4']

# Store results
blink_count = []  # will now be list of lists: per-channel counts
z_score = []      # will be list of 2D arrays: [n_channels, n_times]

# Process each epoch
for ep in all_epochs:
    bc, zs = count_eye_blinks_per_channel(
        eeg_epochs=[ep],  # Pass as list of one epoch
        sfreq=eeg_fs,
        channel_names=ch_names,
        frontal_channels=frontal_ch,
        )
    blink_count.append(bc[0])  # bc[0] is list of blink counts per channel
    z_score.append(zs[0])      # zs[0] is array of shape [n_channels, n_times]

print(blink_count)

In [ ]:
# Get the first epoch for the fourth file (index 3), first stimulus (index 0)
epoch = epochs_organized[9][0]  # shape: (n_epochs, n_channels, n_samples)
single_epoch = epoch[1]         # shape: (16, 1279)

# Average Fz and F4 (channels 0 and 1)
avg_signal = np.mean(single_epoch[[0, 1], :], axis=0)  # shape: (1279,)

# Define segments (assuming 256 Hz sampling rate)
segments = [(0, 512), (512, 1024), (1024, 1279), (0, 1279)]

# Compute global min and max across all segments for y-axis limits
y_min = min(np.min(avg_signal[start:end]) for start, end in segments)
y_max = max(np.max(avg_signal[start:end]) for start, end in segments)

# Create stacked plots
fig, axs = plt.subplots(4, 1, figsize=(10, 8), sharex=False)

titles = [
    "First 2 seconds (0–512 samples)",
    "Next 2 seconds (512–1024 samples)",
    "Final segment (1024–1279 samples)",
    "Entire Epoch (0–1279 samples)"
]

for i, (start, end) in enumerate(segments):
    color = 'tab:red' if i == 3 else 'tab:blue'  # Red for full epoch, blue for others
    x_vals = np.arange(start, end)               # Correct x-values based on segment
    y_vals = avg_signal[start:end]
    
    axs[i].plot(x_vals, y_vals, color=color)
    axs[i].set_title(titles[i])
    axs[i].set_xlabel("Time (samples)")
    axs[i].set_ylabel("Amplitude (µV)")
    axs[i].set_ylim(y_min, y_max)
    axs[i].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
participant = 0
stim_label = "Contrast1Size2"
epoch_idx = 0

# Get the z-scored data for one epoch
z_epoch = z_scores[participant][stim_label][epoch_idx]  # shape: (n_channels, n_samples)
n_samples = z_epoch.shape[1]

colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta', 'brown', 'gray', 'olive']

plt.figure(figsize=(12, 6))

# Plot each channel
for ch_idx in range(z_epoch.shape[0]):
    plt.plot(z_epoch[ch_idx], color=colors[ch_idx % len(colors)], label=f'Ch {ch_idx}')

# Horizontal threshold lines
plt.axhline(2, color='black', linestyle='--', label='Z=±2 Threshold')
plt.axhline(-2, color='black', linestyle='--')

# Vertical lines every 51 samples
for x in range(0, n_samples, 51):
    plt.axvline(x, color='blue', linestyle=':', linewidth=0.8)

plt.title(f"Participant {participant} - Stimulus: {stim_label} - Epoch {epoch_idx}")
plt.xlabel("Sample")
plt.ylabel("Z-score")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
